<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing-Data" data-toc-modified-id="Preprocessing-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing Data</a></span><ul class="toc-item"><li><span><a href="#Split-Intro-Train-and-Test" data-toc-modified-id="Split-Intro-Train-and-Test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Split Intro Train and Test</a></span></li></ul></li><li><span><a href="#Defining-NN-Arquitecture" data-toc-modified-id="Defining-NN-Arquitecture-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Defining NN Arquitecture</a></span></li><li><span><a href="#Training-and-Testing-NN" data-toc-modified-id="Training-and-Testing-NN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training and Testing NN</a></span></li></ul></div>

# Breast Cancer Classsification

In this notebook we ran a neural network with a multilayer perceptron arquitecture in order to classify if a person does have or not breast cancer.

In [45]:
# Import the required libraries
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import StandardScaler
print(f'TF VERSION: {tf.__version__}')

TF VERSION: 2.4.1


## Preprocessing Data

In [46]:
# Read breast cancer data
data = pd.read_csv('../Data/BreastCancer/data.csv')
data = data.loc[:, ~data.columns.str.contains('Unnamed')]
data.columns = data.columns.str.upper()
data.drop(columns={'ID'}, inplace=True)

In [47]:
data.head(n=5)

,DIAGNOSIS,RADIUS_MEAN,TEXTURE_MEAN,PERIMETER_MEAN,AREA_MEAN,SMOOTHNESS_MEAN,COMPACTNESS_MEAN,CONCAVITY_MEAN,CONCAVE POINTS_MEAN,SYMMETRY_MEAN,...,RADIUS_WORST,TEXTURE_WORST,PERIMETER_WORST,AREA_WORST,SMOOTHNESS_WORST,COMPACTNESS_WORST,CONCAVITY_WORST,CONCAVE POINTS_WORST,SYMMETRY_WORST,FRACTAL_DIMENSION_WORST
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### Split Intro Train and Test

In [48]:
# Split into train test
x , y = data.loc[:, data.columns != ('DIAGNOSIS')], data.loc[:,'DIAGNOSIS']
dict_str_int = {'M': 0, 'B': 1}; dict_int_str = {'M': 0, 'B': 1}
y = y.map(dict_str_int)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1608, train_size=0.75)

# Standardize predictors
s_e = StandardScaler()
x_train_std = s_e.fit_transform(x_train)
x_test_std = s_e.transform(x_test)

## Defining NN Arquitecture

In [49]:
# Neural Network 
nn_model = Sequential()
nn_model.add(Dense(25, input_dim=x_train.shape[1], activation='relu', name='Dense_1'))
nn_model.add(Dense(50, input_dim=x_train.shape[1], activation='relu', name='Dense_2'))
nn_model.add(Dropout(0.5, name='Dropout_1'))
nn_model.add(Dense(50, input_dim=x_train.shape[1], activation='relu', name='Dense_3'))
nn_model.add(Dense(25, input_dim=x_train.shape[1], activation='relu', name='Dense_4'))
nn_model.add(Dropout(0.5, name='Dropout_2'))
nn_model.add(Dense(1, activation='sigmoid', name='Dense_Output'))

In [50]:
# Summary of nn_model
nn_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 25)                775       
_________________________________________________________________
Dense_2 (Dense)              (None, 50)                1300      
_________________________________________________________________
Dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
Dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
Dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
Dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
Dense_Output (Dense)         (None, 1)                

In [51]:
# Define loss, optimizator, metrics, etc
nn_model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                 optimizer='rmsprop')

## Training and Testing NN

In [52]:
# Fit the model and meanwhile print train and validation metrics
nn_model.fit(x=x_train_std, y=y_train, batch_size=32, epochs=5, validation_data=(x_test_std, y_test))

Epoch 1/5
14/14 [==============================] - 1s 11ms/step - loss: 0.6319 - accuracy: 0.6754 - val_loss: 0.4083 - val_accuracy: 0.9091
Epoch 2/5
14/14 [==============================] - 0s 3ms/step - loss: 0.4639 - accuracy: 0.7798 - val_loss: 0.2633 - val_accuracy: 0.9510
Epoch 3/5
14/14 [==============================] - 0s 3ms/step - loss: 0.3301 - accuracy: 0.8788 - val_loss: 0.1677 - val_accuracy: 0.9510
Epoch 4/5
14/14 [==============================] - 0s 3ms/step - loss: 0.2611 - accuracy: 0.9139 - val_loss: 0.1232 - val_accuracy: 0.9580
Epoch 5/5
14/14 [==============================] - 0s 3ms/step - loss: 0.1843 - accuracy: 0.9372 - val_loss: 0.1018 - val_accuracy: 0.9650


Neural Network learns pretty well and does not overfit, reaching an approx of 96.50% of accuracy in breast cancer diagnosis!

# End